In [108]:
using LinearAlgebra

In [109]:
function reverse_gauss(A::AbstractMatrix{T}, b::AbstractVector{T}) where T
    x = similar(b)
    N = size(A, 1)
    @inbounds for k in 0:N-1
    x[N-k] = (b[N-k] - sumprod(@view(A[N-k,N-k+1:end]), @view(x[N-k+1:end]))) / A[N-k,N-k]
    end
    return x
end

reverse_gauss (generic function with 1 method)

In [110]:
@inline
function sumprod(A::AbstractVector{T}, B::AbstractVector{T}) where T
    s = T(0)
    @inbounds for i in eachindex(A)
        s = fma(A[i], B[i], s) 
    end
    return s
end

sumprod (generic function with 1 method)

In [111]:
function random_upper_triangular(N::Integer, T::Type)
    A = randn(T, N, N)
    _, A = lu(A)
    return A
end

random_upper_triangular (generic function with 1 method)

In [112]:
Afloat = random_upper_triangular(6, Float64)
bfloat = randn(Float64, 6)
xfloat = reverse_gauss(Afloat, bfloat)

6-element Vector{Float64}:
  0.6794387758415364
 -0.6373962904480566
 -0.6333802259650874
  0.33687745102807337
  0.4140245415991381
 -0.031028730538074218

In [113]:
res_float = bfloat - (Afloat*xfloat)

6-element Vector{Float64}:
 1.6653345369377348e-16
 0.0
 0.0
 0.0
 0.0
 0.0

In [114]:
all(i -> i < 10e-10, res_float)

true

In [115]:
A_bigfloat = random_upper_triangular(6, BigFloat)
b_bigfloat = randn(BigFloat, 6)
x_bigfloat = reverse_gauss(A_bigfloat, b_bigfloat)

6-element Vector{BigFloat}:
 -1.938715531048074156826355436232566136496056593506387003226076868031898302307022
  0.5622349427339233192789851929171229434449965601738707422245920707310169522790507
 -0.3285100902370761641681715081964731513964924796801659625361543696418235911975907
 -1.04452838191765474095891992320725025590271499692022171863130077862611723234693
  0.8192081094946912139569400572794064406069717275566664169856666582669976350452333
 -1.393307449596810310373357397312281230670001784910857474325520677636744868677073

In [116]:
res_big_float = b_bigfloat - (A_bigfloat*x_bigfloat)

6-element Vector{BigFloat}:
  0.0
  4.318084277547222312693175931400199785558000182218140692511851735084295901581214e-78
 -1.727233711018888925077270372560079914223200072887256277004740694033718360632485e-77
 -3.454467422037777850154540745120159828446400145774512554009481388067436721264971e-77
  0.0
  0.0

In [117]:
all(i -> i < 10e-10, res_big_float)

true

In [118]:
function transform_to_steps!(A::AbstractMatrix; epsilon = 1e-7)
    @inbounds for k ∈ 1:size(A, 1)
        try
        Δk = leading_element(@view(A[k:end, k]))
        Δk > 1 && swap!(@view(A[k, k:end]), @view(A[k+Δk-1, k:end]))
        for i ∈ k+1:size(A,1)
            t = A[i,k]/A[k,k]
            @. @views A[i,k:end] = A[i,k:end] - t * A[k,k:end]
        end
        catch
            continue
        end
    end
    return A
end

transform_to_steps! (generic function with 1 method)

In [119]:
@inline
function swap!(A,B)
    @inbounds for i in eachindex(A)
        A[i], B[i] = B[i], A[i]
    end
end

swap! (generic function with 1 method)

In [120]:
function leading_element(a::AbstractVector{T}) where T
    absval, k = findmax(abs, a)
    approx_iszero(absval) && throw("Вырожденая матрица")
    return k
end

function leading_element(a::AbstractVector{Rational{T}}) where T
    k = findfirst(x -> !iszero(x), a)
    isnothing(k) && throw("Матрица вырожденная")
    return k
end

leading_element (generic function with 2 methods)

In [121]:
function gauss(A::AbstractMatrix{T}, b::AbstractVector{T}) where T
    Ab = hcat(A, b)
    transform_to_steps!(Ab)

    return reverse_gauss(@view(Ab[1:end, 1:end-1]), @view(Ab[1:end, end]))
end

gauss (generic function with 1 method)

In [122]:
A = randn(1000, 1000)
b = randn(1000)

1000-element Vector{Float64}:
 -0.444328700119787
 -0.49600737757426083
  0.20764749463570215
  0.5694997858834351
  0.21186016617772388
  0.5401818426022322
 -0.7659776558533277
 -0.0774407696567612
  0.6525918425978452
  1.804089843175802
 -0.3985212105229327
  0.23687232409746703
 -0.43138787184578925
  ⋮
  0.515093264880635
  1.0620974220777921
  0.22123830015643245
  0.39156069724551207
  1.3858463512619483
  1.293467572312447
  1.5933609486108058
  0.2232008263208839
  0.0068286112801767655
  0.5466835602692395
  1.1907924861904178
 -0.8471155723142573

In [124]:
@time x = gauss(A, b)

  0.207212 seconds (295.97 k allocations: 27.020 MiB, 2.29% gc time, 97.96% compilation time)


1000-element Vector{Float64}:
  4.282167446614522e294
 -7.509152845752829e294
  6.755604634382423e293
 -2.212800206227889e293
 -7.045629631928021e293
 -5.010740320737845e293
  1.5555318654912247e293
 -2.758239121401147e292
  2.149172420022001e292
 -2.824980029602101e292
  3.623159962256777e292
  8.362187908234967e292
  5.2166218967936675e292
  ⋮
 48.93136396083188
 18.781757592344913
  1.3084605373323135
  1.3411774958435416
 -0.22878200590538164
 -3.40817889134074
  2.5897678953316245
 -4.717053098976499
  0.13903968855139018
  3.1587353407755656
  2.7728379438817186
  0.45458691609464186

In [125]:
function matrix_rank(A::AbstractMatrix{T}, eps=10e-7) where T
    step_view = copy(A)
    transform_to_steps!(step_view, epsilon=eps)
    count = 0
    for i in 1:size(A)[1]
        flag = false
        for j in 1:size(A)[2]
            flag = flag || abs(step_view[i, j]) >= eps
        end
        if flag == true count += 1 end
    end
    return count
end

matrix_rank (generic function with 2 methods)

In [126]:
A = randn(1000, 1000)

1000×1000 Matrix{Float64}:
  0.839179    0.0762298  -0.909665   …  -1.21981    1.19853    -1.0777
 -1.92395    -0.99572     0.513521      -0.778066   1.73764    -0.398779
 -2.33498    -0.495612    0.355603       2.00505   -0.816358    1.06782
  1.07098     1.39691     0.117308      -0.384464  -0.0820219  -0.652774
 -1.41918     1.20805    -0.29558       -1.7935    -0.136123   -0.424431
  0.37003    -1.33826     1.6245     …  -0.508183  -0.118291   -0.462712
  0.482546    0.291641    0.0364762     -0.568085   1.39944    -1.32904
 -0.0758109   1.24551     0.203815       1.79711   -0.119298   -1.17923
  1.10862     0.381571    0.0464915      0.429376  -0.35997    -0.609551
  0.825368   -0.319206    0.641752      -0.289603   2.22089     0.17825
  1.56791    -0.307694   -1.38544    …  -0.283528  -0.0820811  -0.536946
 -1.82065     1.19002    -0.230847      -0.134795  -0.634479    0.269914
 -0.0888623   0.951196   -0.762308      -0.566894  -1.86048     0.957172
  ⋮                           

In [127]:
matrix_rank(A) == rank(A)

true

In [128]:
function det_steps(A::AbstractMatrix{T}, eps=10e-9) where T
    if size(A)[1] != size(A)[2] throw("Количество строк и столбцов должно быть одинаковое") end

    step_view = copy(A)
    transform_to_steps!(step_view, epsilon=eps)
    val = one(T)
    for i in 1:size(A)[1]
        val *= step_view[i, i]
    end
    return val
end

det_steps (generic function with 2 methods)

In [129]:
A = randn(3, 3)

3×3 Matrix{Float64}:
  0.506693   0.497104   1.5296
  2.66793    1.48959    0.143433
 -0.551142  -0.631689  -1.41508

In [130]:
cond(A)

42.033626472454095

In [131]:
det_steps(A) - det(A)

-0.5612650737095761